In [2]:
library(tidyverse)

── Attaching core tidyverse packages ───────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
credits = read_csv("credits.csv")
titles = read_csv("titles.csv")

Rows: 77801 Columns: 5
── Column specification ─────────────────────────────────────────────────────────────
Delimiter: ","
chr (4): id, name, character, role
dbl (1): person_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 5850 Columns: 15
── Column specification ─────────────────────────────────────────────────────────────
Delimiter: ","
chr (8): id, title, type, description, age_certification, genres, production...
dbl (7): release_year, runtime, seasons, imdb_score, imdb_votes, tmdb_popula...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# data analisys 1
titles %>% filter(type == "SHOW" & str_detect(genres, "action") & imdb_score > 8.9) %>% arrange(desc(imdb_score))

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ts3371,Avatar: The Last Airbender,SHOW,"In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.",2005,TV-Y7,24,"['scifi', 'animation', 'action', 'family', 'fantasy']",['US'],3,tt0417299,9.3,303666,56.915,8.700
ts32835,Hunter x Hunter,SHOW,"Twelve-year-old Gon Freecss one day discovers that the father he had always been told was dead was alive. His Father, Ging, is a Hunter — a member of society's elite with a license to go anywhere or do almost anything. Gon, determined to follow in his father's footsteps, decides to take the Hunter Examination and eventually find his father to prove himself as a Hunter in his own right. But on the way, he learns that there is more to becoming a Hunter than previously thought, and the challenges that he must face are considered the toughest in the world.",2011,TV-14,23,"['action', 'animation', 'comedy', 'fantasy']",['JP'],3,tt2098220,9.0,92162,178.265,8.800
ts20682,Attack on Titan,SHOW,"Several hundred years ago, humans were nearly exterminated by Titans. Titans are typically several stories tall, seem to have no intelligence, devour human beings and, worst of all, seem to do it for the pleasure rather than as a food source. A small percentage of humanity survived by walling themselves in a city protected by extremely high walls, even taller than the biggest Titans. Flash forward to the present and the city has not seen a Titan in over 100 years. Teenage boy Eren and his foster sister Mikasa witness something horrific as the city walls are destroyed by a Colossal Titan that appears out of thin air. As the smaller Titans flood the city, the two kids watch in horror as their mother is eaten alive. Eren vows that he will murder every single Titan and take revenge for all of mankind.",2013,TV-MA,24,"['action', 'scifi', 'animation', 'horror', 'drama', 'fantasy']",['JP'],4,tt2560140,9.0,325381,85.661,8.600
ts222333,Arcane,SHOW,"Amid the stark discord of twin cities Piltover and Zaun, two sisters fight on rival sides of a war between magic technologies and clashing convictions.",2021,TV-14,41,"['scifi', 'action', 'drama', 'animation', 'fantasy']",['US'],1,tt11126994,9.0,186899,106.450,9.081


In [5]:
# data analisys 2

# for countries: japan(JP) and france (FR)

filter_countries = function(production_countries){
    # make sure its not a coproduction
    
    cond_1 = str_detect(production_countries,"JP") | str_detect(production_countries,"FR")
    cond_2 = nchar(production_countries) == 6
    return(cond_1 & cond_2)
}

#filter_countries(titles$production_countries)
titles  %>% filter(filter_countries(production_countries)) %>%  # only movies made in JP or FR (no copro)
            group_by(production_countries) %>%  # group by where it was made
            arrange(desc(imdb_score), desc(imdb_votes), desc(tmdb_score)) %>% # sort
            slice_head(n=2) # top 2 of each county

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ts37866,Call My Agent!,SHOW,"At a top Paris talent firm, agents scramble to keep their star clients happy—and their business afloat—after an unexpected crisis.",2015,TV-MA,55,"['comedy', 'drama', 'european']",['FR'],4,tt4277922,8.3,14061,20.420,7.900
ts81526,November 13: Attack on Paris,SHOW,"Survivors and first responders share personal stories of anguish, kindness and bravery that unfolded amid the Paris terror attacks of Nov. 13, 2015.",2018,TV-MA,54,['documentation'],['FR'],1,tt8194716,8.3,3137,4.169,8.200
ts20682,Attack on Titan,SHOW,"Several hundred years ago, humans were nearly exterminated by Titans. Titans are typically several stories tall, seem to have no intelligence, devour human beings and, worst of all, seem to do it for the pleasure rather than as a food source. A small percentage of humanity survived by walling themselves in a city protected by extremely high walls, even taller than the biggest Titans. Flash forward to the present and the city has not seen a Titan in over 100 years. Teenage boy Eren and his foster sister Mikasa witness something horrific as the city walls are destroyed by a Colossal Titan that appears out of thin air. As the smaller Titans flood the city, the two kids watch in horror as their mother is eaten alive. Eren vows that he will murder every single Titan and take revenge for all of mankind.",2013,TV-MA,24,"['action', 'scifi', 'animation', 'horror', 'drama', 'fantasy']",['JP'],4,tt2560140,9.0,325381,85.661,8.600
ts11313,DEATH NOTE,SHOW,"Light Yagami is an ace student with great prospects—and he’s bored out of his mind. But all that changes when he finds the Death Note, a notebook dropped by a rogue Shinigami death god. Any human whose name is written in the notebook dies, and Light has vowed to use the power of the Death Note to rid the world of evil. But will Light succeed in his noble goal, or will the Death Note turn him into the very thing he fights against?",2006,TV-14,24,"['animation', 'thriller', 'scifi', 'drama', 'fantasy', 'crime']",['JP'],1,tt0877057,9.0,309445,120.545,8.664


In [22]:
# Data analisys 3
genre = "scifi"



# get data of every scifi movie 
merged_data <- titles %>%
  full_join(credits, by = "id") %>%
    filter(type == "MOVIE", str_detect(genres,genre), !is.na(imdb_score))


# first we get both the avarage score and the total appearances of each actor 
actor_data <- merged_data %>%
  group_by(person_id) %>%
  summarize(
    name = first(name),
    total_appearances = n(),
    avg_score = mean(imdb_score)
  )

# then we normalize both metrics, so that the imdb score doesnt define the final rank alone
# and combine both into a weighted rank
actor_data <- actor_data %>%
  mutate(
    normalized_score = ((avg_score - mean(avg_score)) / sd(avg_score)),
    normalized_appearances = (total_appearances - mean(total_appearances)) / sd(total_appearances) 
  ) %>%
 mutate(
    combined_rank = 0.6 * normalized_appearances + 0.4 * normalized_score
  )


actor_data %>%
  arrange(desc(combined_rank)) %>%
  head(3)





person_id,name,total_appearances,avg_score,normalized_score,normalized_appearances,combined_rank
<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
6821,Fred Tatasciore,7,6.271429,0.1709566,18.15623,10.962119
75908,Kenjiro Tsuda,5,6.700000,0.5522231,12.02859,7.438046
4889,Ken Jeong,5,6.220000,0.1252046,12.02859,7.267238


In [8]:
# Data analisys 4

# first we join the tables since one holds the score and the other holds the actors info
merged_data <- titles %>%
  full_join(credits, by = "id") 


actor_scores <- merged_data %>%    
  # make sure our table only holds data about actors
  group_by(person_id, name) %>% 
    filter(role == "ACTOR") %>%    
  # gets the avarage score of every actor
  summarise(average_imdb_score = mean(imdb_score, na.rm = TRUE), .groups = "drop")


# top 3 best actors
actor_scores %>%
  arrange(desc(average_imdb_score)) %>%                     
  head(3)         

# top 3 worst actors
actor_scores %>%
  filter(!is.na(average_imdb_score)) %>%                     
  arrange(average_imdb_score) %>%                            
  head(3)  

merged_data


person_id,name,average_imdb_score
<dbl>,<chr>,<dbl>
37752,Anna Gunn,9.5
1303,Jessie Flower,9.3
21033,Zach Tyler,9.3


person_id,name,average_imdb_score
<dbl>,<chr>,<dbl>
596682,Mansoor Alfeeli,1.5
1430247,Ana Druzhynina,1.5
1431133,Saga Alyasery,1.5


id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
ts300399,Five Came Back: The Reference Films,SHOW,"This collection includes 12 World War II-era propaganda films — many of which are graphic and offensive — discussed in the docuseries ""Five Came Back.""",1945,TV-MA,51,['documentation'],['US'],1,NA,NA,NA,0.600,NA,NA,NA,NA,NA
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,32267,Peter Boyle,Wizard,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,519612,Leonard Harris,Senator Charles Palantine,ACTOR
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],NA,tt0075314,8.2,808582,40.965,8.179,29068,Diahnne Abbott,Concession Girl,ACTOR
